In [1]:
import pandas as pd
import re
import nltk
# nltk.download('vader_lexicon')
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Read the data
news = pd.read_csv('../textData/bac_news.csv')

In [6]:
news.head()

,Unnamed: 0,id,ticker,title,category,content,release_date,provider,url,article_id
0,41710,263225,BAC,U S Investors Go All In On Equity ETFs,opinion,at the end of last week analysts at bank of am...,2016-12-20,Rupert Hargreaves,https://www.investing.com/analysis/u.s.-invest...,200170359
1,41711,263226,BAC,It s The Big Banks Time To Shine,opinion,a couple of days after donald trump was electe...,2016-12-23,Kenneth Ashe,https://www.investing.com/analysis/it's-the-bi...,200170694
2,41712,263227,BAC,7 Reasons To Invest In Bank Of America Heading...,opinion,there are a few times in the life of an invest...,2016-12-29,Kenneth Ashe,https://www.investing.com/analysis/seven-reaso...,200171060
3,41713,263228,BAC,Outlook 2017 Part II Stocks Bonds Inflatio...,opinion,without question was the year of the big surpr...,2016-12-30,Investing.com,https://www.investing.com/analysis/outlook-201...,200170791
4,41714,263229,BAC,Financial Crash Fallout Continues U S Sues B...,opinion,if you saw your barclays lon barc stock take a...,2017-01-01,Modest Money,https://www.investing.com/analysis/financial-c...,200171173


In [13]:
news.rename({'release_date': 'date'}, axis=1, inplace=True)
news_clean = news[['content', 'date']]
news_clean

# Generate sentiment score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

texts = news_clean['content'].tolist()

output_score = []
for text in texts:
  output_score.append(sid.polarity_scores(str(text))["compound"])

news_clean.loc[:, 'Sentiment_Score'] = output_score

news_clean.sort_values(by = 'date', inplace = True)

# Pure sentiment score!!!
news_clean.tail()

,content,date,Sentiment_Score
1064,by andreas kr ner and karen freifeld frankfurt...,2016-12-19,-0.9337
13,another bullish voice given that many of the c...,2016-12-19,0.9780
12,by eli wright following the fed rate hike this...,2016-12-19,0.9984
1065,by joshua franklin oliver hirt and karen freif...,2016-12-19,0.9675
0,at the end of last week analysts at bank of am...,2016-12-20,0.0094
1068,by sarah n lynch washington reuters morgan sta...,2016-12-20,-0.7759
1067,by simon jessop and lawrence white london reut...,2016-12-20,0.9726
1066,european stocks registered mixed trade on tues...,2016-12-20,0.9923
14,the federal reserve announced last week it wou...,2016-12-20,0.9978
1069,by jonathan saul and andrew macaskill london r...,2016-12-21,-0.9538


In [15]:
news_clean['Negative'] = [0] * len(news_clean)
news_clean['Neutral'] = [0] * len(news_clean)
news_clean['Positive'] = [0] * len(news_clean)

for i in range(len(news_clean)):
    if news_clean.Sentiment_Score[i] < 0:
        news_clean.Negative[i] = 1
    elif news_clean.Sentiment_Score[i] > 0:
        news_clean.Positive[i] = 1
    else:
        news_clean.Neutral[i] = 1
        
news_clean.tail(20)

,content,date,Sentiment_Score,Negative,Neutral,Positive
1064,by andreas kr ner and karen freifeld frankfurt...,2016-12-19,-0.9337,1,0,0
13,another bullish voice given that many of the c...,2016-12-19,0.9780,0,0,1
12,by eli wright following the fed rate hike this...,2016-12-19,0.9984,0,0,1
1065,by joshua franklin oliver hirt and karen freif...,2016-12-19,0.9675,0,0,1
0,at the end of last week analysts at bank of am...,2016-12-20,0.0094,0,0,1
1068,by sarah n lynch washington reuters morgan sta...,2016-12-20,-0.7759,1,0,0
1067,by simon jessop and lawrence white london reut...,2016-12-20,0.9726,0,0,1
1066,european stocks registered mixed trade on tues...,2016-12-20,0.9923,0,0,1
14,the federal reserve announced last week it wou...,2016-12-20,0.9978,0,0,1
1069,by jonathan saul and andrew macaskill london r...,2016-12-21,-0.9538,1,0,0


In [22]:
news_clean_3 = news_clean[['date', 'Negative', 'Neutral', 'Positive']]
news_clean_3 = news_clean_3.groupby('date').mean()
news_clean_3.tail(20)

,Negative,Neutral,Positive
date,,,
2016-12-06,0.000000,0.000000,1.000000
2016-12-07,0.000000,0.000000,1.000000
2016-12-08,0.181818,0.000000,0.818182
2016-12-09,0.000000,0.000000,1.000000
2016-12-12,0.000000,0.000000,1.000000
2016-12-13,0.250000,0.000000,0.750000
2016-12-14,1.000000,0.000000,0.000000
2016-12-15,0.333333,0.000000,0.666667
2016-12-16,0.333333,0.333333,0.333333


In [23]:
news_clean_3.to_csv('news_vader_avg.csv')